# Understanding Methane Spectra

**Summary**  

In this notebook, we'll examine a EMIT L1B At-Sensor Calibrated Radiance ([EMITL1BRAD](https://lpdaac.usgs.gov/products/emitl1bradv001/)) scene to visualize the spectra for a sample point inside a methane plume, and two sample points outside of methane plumes. To highlight the spectral differences we will look at an in-plume divided by out-of-plume ratio to get an example of the spectral signature or fingerprint of methane and visually compare this to the target methane signature 
or modeled methane transmission.

**Background**

Methane is major contributor to atmospheric radiative forcing because its more efficient at trapping radiation than other greenhouse gases, like carbon dioxide. Because the lifetime of methane in the atmosphere is only about 10 years, reducing methane emissions offers an effective way to curb anthropogenic contributions to atmospheric radiative forcing.

The EMIT instrument is an imaging spectrometer that measures light in visible and infrared wavelengths. These measurements display unique spectral signatures that correspond to chemical composition. Although the primary mission focuses on mapping the mineral composition of Earth's surface, the EMIT instrument has also been used to successfully map methane point source emissions within its target mask using these unique spectral signatures. More details about EMIT and its associated products can be found in the **README.md** and on the [EMIT website](https://earth.jpl.nasa.gov/emit/).

The EMITL1BRAD product provides at-sensor calibrated radiance values along with observation data in a spatially raw, non-orthocorrected format. This product is used to estimate the EMIT L2B Methane Enhancement Data ([EMITL2BCH4ENH](https://lpdaac.usgs.gov/products/emitl2bch4enhv001/)) product. The EMITL2BCH4ENH product is created using an adaptive matched filter to search each pixel's radiance spectrum for deviations that are characteristic of methane's absorption spectrum, and is only delivered to the LP DAAC for scenes where plume complexes have been identified. To reduce the risk of false positives, all EMITL2BCH4ENH data undergo a manual review (or identification and confirmation) process before being designated as a plume complex. For more information on the identification and manual review process, see Section 4.2.2 of the [EMIT GHG Algorithm Theoretical Basis Document (ATBD)](https://lpdaac.usgs.gov/documents/1696/EMIT_GHG_ATBD_V1.pdf). 


**References**

Andrew K. Thorpe et al., Attribution of individual methane and carbon dioxide emission sources using EMIT observations from space. Sci. Adv.9, eadh2391 (2023). DOI:[10.1126/sciadv.adh2391](https://www.science.org/doi/10.1126/sciadv.adh2391)

**Requirements** 
 - Set up Python Environment - See **setup_instructions.md** in the `/setup/` folder
 - NASA Earthdata Login Account. [Sign Up](https://urs.earthdata.nasa.gov/users/new)
 - GitHub username for 2i2c Cloud Workspace Access. Create a [GitHub account](https://github.com/join). Complete the form to request access to the 2i2c Cloud Workspace [here](). #TODO - Add link to newest form


**Data Used** 
 - [EMIT L1B At-Sensor Calibrated Radiance and Geolocation Data (EMITL1BRAD)](https://lpdaac.usgs.gov/products/emitl1bradv001/)
 - [EMIT L2B Methane Enhancement Data (EMITL2BCH4ENH)](https://lpdaac.usgs.gov/products/emitl2arflv001/)


**Learning Objectives** 
 - Open and orthorectify an EMITL1BRAD scene
 - Open and visualize the scene and point data
 - Extract point data using coordinates in a .csv file
 - Visualize spectral fingerprint of methane and target methane spectra

**Tutorial Outline**  

1. [**Setup**](#setup)
2. [**Opening EMIT Data**](#opening)
3. [**Extracting Point Data**](#extracting_points)
4. [**Methane Spectral Signature**](#spectral_signature)

## 1. Setup<a id='setup'></a>

Import the necessary Python libraries.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from osgeo import gdal
import earthaccess
import rasterio as rio
import rioxarray as rxr
import holoviews as hv
import hvplot
import hvplot.xarray
import hvplot.pandas
from skimage import exposure

sys.path.append('../modules/')
from emit_tools import emit_xarray

Download the EMIT L1B Radiance and L2B Methane Enhancement products for the scene we're going to look at.

In [ ]:
urls = ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL1BRAD.001/EMIT_L1B_RAD_001_20220815T042838_2222703_003/EMIT_L1B_RAD_001_20220815T042838_2222703_003.nc',
        'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2BCH4ENH.001/EMIT_L2B_CH4ENH_001_20220815T042838_2222703_003/EMIT_L2B_CH4ENH_001_20220815T042838_2222703_003.tif']

# Authenticate and create an https session
earthaccess.login(persist=True)
fs = earthaccess.get_requests_https_session()

for url in urls:
# Retrieve granule asset ID from URL (to maintain existing naming convention)
    granule_asset_id = url.split('/')[-1]
    # Define Local Filepath
    fp = f'../../data/{granule_asset_id}'
    # Download the Granule Asset if it doesn't exist
    if not os.path.isfile(fp):
        with fs.get(url,stream=True) as src:
            with open(fp,'wb') as dst:
                for chunk in src.iter_content(chunk_size=64*1024*1024):
                    dst.write(chunk)

Set the filepath for the radiance and methane enhancement files.

In [ ]:
rad_fp = '../../data/EMIT_L1B_RAD_001_20220815T042838_2222703_003.nc'
enh_fp = '../../data/EMIT_L2B_CH4ENH_001_20220815T042838_2222703_003.tif'

## 2. Opening EMIT Data<a id='opening'></a>

The EMIT L1B At-Sensor Radiance data is distributed in a non-orthorectified spatially raw netCDF4 (.nc) format consisting of the data and its associated metadata. Inside the L1B file, there are 3 groups. 

1. The root group that can be considered the main dataset contains the radiance data described by the downtrack, crosstrack, and bands dimensions.  
2. The sensor_band_parameters group containing the wavelength center and the full-width half maximum (FWHM) of each band.  
3. The location group contains latitude and longitude values at the center of each pixel described by the crosstrack and downtrack dimensions, as well as a geometry lookup table (GLT) described by the ortho_x and ortho_y dimensions. The GLT is an orthorectified image (EPSG:4326) consisting of 2 layers containing downtrack and crosstrack indices. These index positions allow us to quickly project the raw data onto this geographic grid.  

This data can be opened using the `netCDF4` and `xarray` libraries, or utilizing functions within the `emit_tools.py` module to organize them into a flattened (no groups) `xarray.Dataset` object. For this notebook, we will use functions from `emit_tools.py` to simplify working with the data. For more about the structure and use of `netCDF4` and `xarray` please see the [Exploring EMIT L2A Reflectance Jupyter Notebook](https://github.com/nasa/EMIT-Data-Resources/blob/main/python/tutorials/Exploring_EMIT_L2A_Reflectance.ipynb).

Open the radiance file using the `emit_xarray` function from the `emit_tools.py` module and orthorecitify it.

In [ ]:
rad = emit_xarray(rad_fp, ortho=True)
rad

In [ ]:
rad = rad.where(rad.wavelengths > 2100, drop=True)

The EMIT L2B Methane Enhancement Data represent an enhancement above background methane concentration for a 1 meter layer in parts-per-million (ppm) meter (m). These units are used rather than ppm because we are unable to measure the vertical extent of plumes. This data is distributed as a single band cloud-optimized geotiff (COG) and it has been orthocorrected. We can open this using the `rioxarray` library to place the data in an `xarray.DataArray` object.

Open the methane enhancement geotiff file using `rioxarray` and squeeze the `band` dimension to remove the extra dimension so our array will only have 2 dimensions.

In [ ]:
enh = rxr.open_rasterio(enh_fp).squeeze('band',drop=True) 
enh

## 3. Extracting Point Data <a id='extracting_points'></a>

Open the `.csv` file included in the `/data/` directory as a `pandas.DataFrame`. This file contains the latitude and longitude coordinates for three points of interest. Two that are outside of a methane plume, and one that is inside.

In [ ]:
# Define our Points for In-plume and out-of-plume
points = pd.read_csv('../../data/methane_tutorial/methane_inout_points.csv')
points

Set the index as the `ID` column.

In [ ]:
points = points.set_index(['ID'])
points

Now we can use the `sel` function from `xarray` to extract the radiance data at each point in our dataframe.

In [ ]:
# Extract target spectra from dataset
point_ds = rad.sel(latitude=points.to_xarray().latitude, longitude=points.to_xarray().longitude, method='nearest')

After extracting, we can convert the data to a `pandas.DataFrame` and join it with our 'in-plume' column from the original dataframe.

In [ ]:
point_df = point_ds.to_dataframe().join(points['in-plume'],on=['ID'])
point_df

At this point, we can save the data to a `.csv` file for future use.

In [ ]:
# point_df.to_csv('../../data/methane_tutorial/point_df.csv')

Next, visualize these points on an rgb image generated from the radiance file, and the methane enhancement data, just to get a better idea of where these points are located.

To do this, first create an RGB data array from the radiance file using the `sel` function to select the bands nearest to the desired wavelengths.

In [ ]:
# Create an RGB from Radiance
rgb = rad.sel(wavelengths=[650,560,470], method='nearest')

Next, use a function to rescale the brightness, so this image is easier to see.

In [ ]:
rgb_data = np.nan_to_num(rgb.radiance.data, nan=0)
rgb_data = exposure.rescale_intensity(rgb_data, in_range='image', out_range=(0,1))
rgb.radiance.data = rgb_data

Now that we have the necessary pieces to visualize we can make some spatial visualizations using `hvplot`. For the enhancement data, set the fill value of -9999 to `np.nan` to make it transparent.

In [ ]:
# Create RGB Plot
rgb_map = rgb.hvplot.rgb(x='longitude',y='latitude',bands='wavelengths',title='RGB Radiance', geo=True, crs='EPSG:4326')

In [ ]:
# Create Methane Enhancement Plot
enh.data[enh.data == -9999] = np.nan
methane_map = enh.hvplot.image(x='x',y='y',cmap='viridis', geo=True, crs='EPSG:4326', title='Methane Enhancement', clabel='ppm m', xlabel='Longitude', ylabel='Latitude')

In [ ]:
point_map = point_df.hvplot.points(x='longitude',y='latitude', color='in-plume', cmap='reds', geo=True, crs='EPSG:4326', hover=False, colorbar=False)

We can combine these plots with an `*` operator to overlay them on the same plot. This does require that all are in the same CRS to display properly.

In [ ]:
(rgb_map*point_map) + (methane_map*point_map)

Now lets look at the spectra. 

In [ ]:
point_df.hvplot(x='wavelengths',y='radiance', by=['ID'], frame_height=400, frame_width=600, title = 'Radiance Spectra, ID 0 is in-plume' , xlabel='Wavelength (nm)', ylabel='Radiance (W/m^2/sr/nm)')

## 4. Methane Spectral Signature<a id='spectral_signature'></a>

Create a band ratio of in plume to out of plume spectra. This method can reveal subtle signals, and will isolate the signal coming from methane because the rest of the scene is fairly homogenous.

In [ ]:
in_plume = point_df.loc[point_df['in-plume'] == 1].copy()
out_plume = point_df.loc[point_df['in-plume'] == 0].copy()

In [ ]:
out_plume['band_ratio'] = (in_plume.loc[0,'radiance']/out_plume['radiance'])

As we visualize this we want to focus on the electromagnetic spectrum between 2150 and 2450 nanometers (nm). This is where the strongest methane spectral features are located.

In [ ]:
ratio_plot = out_plume.hvplot(x='wavelengths',y='band_ratio', by=['ID'], frame_height=400, frame_width=400, xlim=(2150,2450), ylim=(0.85,1.0), ylabel='Band Ratio In/Out of Plume', xlabel='Wavelength (nm)')

Open the .csv containing the absorption coefficient of methane. This is what the key spectral features used to identify methane should look like to the EMIT instrument.

In [ ]:
ch4_ac = pd.read_csv('../../data/methane_tutorial/emit20220815t042838_o22703_s000_l1b_dua_spec_ch4.csv')

Create a figure for the absorption coefficient. 

In [ ]:
ac_plot = ch4_ac.hvplot(x='wavelength',y='value', frame_height=400, frame_width=400, line_color='black', line_width=2, xlim=(2150,2450), ylim=(-1.3,0.4), xlabel='Wavelength (nm)', ylabel='Modelled Methane Transmission')

We will plot this along with the in/out of plume band ratio to compare the two. Since `hvplot` does not yet support twin axes plots, we'll have to create a function to do this using some functions from `bokeh`, one of the underlying libraries used by `hvplot`.

In [ ]:
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d

def overlay_hook(plot, element):
    # Adds right y-axis
    p = plot.handles["plot"]
    p.extra_y_scales = {"right": LinearScale()}
    p.extra_y_ranges = {"right": Range1d(-1.3,0)}
    p.add_layout(LinearAxis(y_range_name="right"), "right")

   # find the last line and set it to right
    lines = [p for p in p.renderers if isinstance(p, GlyphRenderer)]
    lines[-1].y_range_name = "right"

# Create Figure
(ratio_plot.opts(legend_position='bottom') * ac_plot.opts(color="k")).opts(hooks=[overlay_hook]).opts(title='in plume/out of plume and absorption coefficient') 

As you can see, after adjusting the y-axes, the curves are very similar and this indicates that the in/out of plume band ratio is a good representation of the methane spectral signature.

## Contact Info:
Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: https://lpdaac.usgs.gov/  
Date last modified: 03-13-2024  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  
